In [ ]:
# | default_exp _helpers.api_docs_helper

In [ ]:
# | export

from typing import *
import types
import os
from pathlib import Path
from inspect import isfunction, isclass, getmembers, getsourcefile, isroutine, ismethod, isclass

import griffe
import yaml
from nbdev.config import get_config

from nbdev_mkdocs._helpers.utils import raise_error_and_exit

In [ ]:
import random
import sys
from subprocess import CalledProcessError
from abc import abstractmethod
import shutil
import unittest.mock
from contextlib import contextmanager
from tempfile import TemporaryDirectory

from nbdev.doclinks import NbdevLookup
from fastcore.basics import patch

from nbdev_mkdocs.mkdocs import prepare, _get_submodule_members, _load_submodules
from nbdev_mkdocs._helpers.utils import set_cwd

In [ ]:
# | export


def _get_symbol_filepath(symbol: Union[types.FunctionType, Type[Any]]) -> Path:
    config = get_config()
    filepath = getsourcefile(symbol)
    return Path(filepath).relative_to( # type: ignore
        filepath.split(f'{config["lib_path"].name}/')[0] # type: ignore
    )

In [ ]:
def sample_test_function():
    pass

sample_test_function.__module__ = "custom_name._components.Sample.sample"

@contextmanager
def mock_getsourcefile():
    with unittest.mock.patch('__main__.getsourcefile') as mock_getsourcefile:
        mock_getsourcefile.return_value = '/Users/username/Dev/nbdev-mkdocs/custom_name/_components/Sample.py'
        yield
        
@contextmanager
def mock_get_config():
    with unittest.mock.patch('__main__.get_config') as mock_get_config:
        mock_get_config.return_value = {"lib_path": Path('/Users/username/Dev/nbdev-mkdocs/custom_name')}
        yield

with mock_getsourcefile():
    with mock_get_config():
        actual = _get_symbol_filepath(sample_test_function)
        expected = Path("custom_name/_components/Sample.py")
        print(actual)
        assert actual == expected

In [ ]:
actual = _get_symbol_filepath(prepare)
expected = Path("nbdev_mkdocs/mkdocs.py")
print(actual)

assert actual == expected

In [ ]:
# | export


def _generate_autodoc(symbol: Union[types.FunctionType, Type[Any]], symbol_path: Path) -> str:
    return f"\n\n::: {os.path.splitext(str(symbol_path).replace('/', '.'))[0]}.{symbol.__name__}\n"

In [ ]:
expected = "\n\n::: custom_name._components.Sample.sample_test_function\n"

actual = _generate_autodoc(symbol=sample_test_function, symbol_path=Path("custom_name/_components/Sample.py"))
print(actual)
assert actual == expected

In [ ]:
# | export


def _add_mkdocstring_header_config(
    autodoc: str, heading_level: int, show_category_heading: bool, is_root_object: bool
) -> str:
    """Adds the mkdocstring header configuration to the autodoc string.

    Args:
        autodoc: The autodoc string to modify.
        heading_level: The base heading level set in the mkdocs config file.
        show_category_heading: The value of the show_category_heading flag set in the mkdocs config file.
        is_root_object: A flag indicating whether the object is the root object.

    Returns:
        The modified autodoc string with the heading level and options.

    """
    if not is_root_object:
        autodoc_header_level = (
            heading_level + 2 if show_category_heading else heading_level + 1
        )
        autodoc += f"    options:\n      heading_level: {autodoc_header_level}\n      show_root_full_path: false\n"
    return autodoc

In [ ]:
autodoc = "\n\n::: nbdev_mkdocs.mkdocs.prepare\n"
actual = _add_mkdocstring_header_config(autodoc=autodoc, heading_level=2, show_category_heading=True, is_root_object=True)

expected = "\n\n::: nbdev_mkdocs.mkdocs.prepare\n"
display(actual)
assert actual == expected

In [ ]:
autodoc = "\n\n::: nbdev_mkdocs.mkdocs.prepare\n"
actual = _add_mkdocstring_header_config(autodoc=autodoc, heading_level=2, show_category_heading=True, is_root_object=False)

expected = """\n\n::: nbdev_mkdocs.mkdocs.prepare
    options:
      heading_level: 4
      show_root_full_path: false
"""
display(actual)
assert actual == expected

In [ ]:
autodoc = "\n\n::: nbdev_mkdocs.mkdocs.prepare\n"
actual = _add_mkdocstring_header_config(autodoc=autodoc, heading_level=2, show_category_heading=False, is_root_object=False)

expected = """\n\n::: nbdev_mkdocs.mkdocs.prepare
    options:
      heading_level: 3
      show_root_full_path: false
"""
display(actual)
assert actual == expected

In [ ]:
# | export


def _is_method(symbol: Union[types.FunctionType, Type[Any]]) -> bool:
    """Check if the given symbol is a method or a property.

    Args:
        symbol: A function or method object to check.

    Returns:
        A boolean indicating whether the symbol is a method.
    """
    return (
        ismethod(symbol)
        or isfunction(symbol)
        or isinstance(symbol, property)
    )

In [ ]:
class MyClass:
    attribute = "Some Attribute"

    def instance_method(self, a, b, c):
        """instance_method documentation"""
        pass

    @property
    def property_attribute(self, a):
        """property_attribute documentation"""
        return self.attribute

    @classmethod
    def class_method(cls, a):
        """class_method documentation"""
        return cls.class_variable

    @staticmethod
    def static_method(x):
        """static_method documentation"""
        pass

    @abstractmethod
    def abstract_method(self, xyz):
        """abstract_method documentation"""
        pass
    
assert _is_method(MyClass.instance_method)
assert _is_method(MyClass.static_method)
assert _is_method(MyClass.class_method)
assert _is_method(MyClass.abstract_method)
assert _is_method(MyClass.property_attribute)
assert not _is_method(MyClass.attribute)

In [ ]:
# | export


def _generate_autodoc_string(
    symbol: Union[types.FunctionType, Type[Any]],
    *,
    heading_level: int,
    show_category_heading: bool,
    is_root_object: bool = True,
) -> str:
    """Generate the autodoc string for the given symbol.

    Args:
        symbol: The symbol to generate the autodoc string for.
        heading_level: The base heading level set in the mkdocs config file.
        show_category_heading: The value of the show_category_heading flag set in the mkdocs config file.
        is_root_object: A flag indicating whether the object is the root object.

    Returns:
        The generated autodoc string with the appropriate heading level and options.

    """
    if isinstance(symbol, property):
        symbol = symbol.fget
    try:
        module = f"{symbol.__module__}.{symbol.__qualname__}"
        parsed_module = griffe.load(module)
        if "raise NotImplementedError()" in parsed_module.source:
            raise KeyError
        autodoc = f"\n\n::: {module}\n"
    except KeyError as e:
        patched_symbol_path = _get_symbol_filepath(symbol)
        autodoc = _generate_autodoc(symbol, patched_symbol_path)

    return _add_mkdocstring_header_config(
        autodoc, heading_level, show_category_heading, is_root_object
    )

In [ ]:
@contextmanager
def add_tmp_path_to_sys_path(dir_):
    dir_ = Path(dir_).absolute().resolve(strict=True)
    original_path = sys.path[:]
    sys.path.insert(0, str(dir_))
    try:
        yield
    finally:
        sys.path = original_path


_mkdocs = """
plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
        - https://docs.python.org/3/objects.inv
        options:
          heading_level: 2
          show_category_heading: false
          show_root_heading: true
          show_signature_annotations: true
          show_if_no_docstring: true
"""

@contextmanager
def create_test_package(package_name: str, module_code: str):
    with TemporaryDirectory() as d:
        my_package_path = Path(d) / package_name
        my_package_path.mkdir(parents=True)

        module_name = "mymodule"
        file_path = my_package_path / f"{module_name}.py"

        with open(file_path, "w", encoding="utf-8") as file:
            file.write(module_code)

        with open((my_package_path / "__init__.py"), "w", encoding="utf-8") as f:
            f.write('__version__ = "0.0.1"')
            
        mkdocs_path = Path(d) / "mkdocs"
        mkdocs_path.mkdir(parents=True)

        with open((mkdocs_path / "mkdocs.yml"), "w", encoding="utf-8") as f:
            f.write(_mkdocs)
            
        yield d

In [ ]:
module_code = '''
from typing import *
from abc import abstractmethod

from fastcore.basics import patch

class FixtureClass:
    """Fixture documentation"""
    
    def instance_method(self):
        """instance_method documentation"""
        return "This is an instance method"
'''

package_name = f"mypackage_{random.randint(0, 1000)}"
with create_test_package(package_name, module_code) as d:
    with add_tmp_path_to_sys_path(d):
        members_with_submodules = _get_submodule_members(package_name)
        symbols = _load_submodules(package_name, members_with_submodules)
        members_list = [y for x, y in getmembers(symbols[0]) if not x.startswith("_") and _is_method(y)]
        
        with set_cwd(d):
            actual = [
                _generate_autodoc_string(m, heading_level=2, show_category_heading=False)
                for m in members_list
            ]
            print(actual)
            d_name = d.split("/")[-1]
            expected = [
                f"\n\n::: {package_name}.mymodule.FixtureClass.abstract_method\n",
                f"\n\n::: {package_name}.mymodule.FixtureClass.class_method\n",
                f"\n\n::: {package_name}.mymodule.FixtureClass.instance_method\n",
                f"\n\n::: {d_name}.{package_name}.mymodule.patched_method_in_same_file\n",
                f"\n\n::: {package_name}.mymodule.FixtureClass.property_attribute\n",
                f"\n\n::: {package_name}.mymodule.FixtureClass.static_method\n",
            ]
#             assert actual == expected, expected

In [ ]:
# | export


def _filter_attributes_in_autodoc(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    """Add symbol attributes to exclude in the autodoc string.

    Args:
        symbol: The symbol for which the filters to be added.

    Returns:
        The autodoc string along with the filters.

    """
    members_list = [
        f'"!^{x}$"'
        for x, y in getmembers(symbol)
        # do not add __init__ to filters else Functions sections will not render in sidenav
        if _is_method(y) and x != "__init__"
    ]
    return f"""    options:
      filters: [{", ".join(members_list)}]"""

In [ ]:
class FixtureClass:
    """Fixture documentation"""
    
    class NestedClass:
        pass
    
    def __init__(self, attribute):
        """__init__ documentation"""
        self.attribute = attribute
        
    def __str__(self):
        """__str__ documentation"""
        return f"MyClass instance with attribute: {self.attribute}"
    
    @property
    def property_attribute(self):
        """property_attribute documentation"""
        return self.attribute
    
    @classmethod
    def class_method(cls):
        """class_method documentation"""
        return cls.class_variable
    
    @staticmethod
    def static_method():
        """static_method documentation"""
        return "This is a static method"
    
    def instance_method(self):
        """instance_method documentation"""
        return "This is an instance method"
    
    @abstractmethod
    def abstract_method(self):
        """abstract_method documentation"""
        pass
    
@patch
def patched_method_in_same_file(self:FixtureClass, s: str) -> None: 
    """I am a patched method in the same file"""
    pass



actual = _filter_attributes_in_autodoc(FixtureClass)
expected = """    options:
      filters: ["!^__str__$", "!^abstract_method$", "!^class_method$", "!^instance_method$", "!^patched_method_in_same_file$", "!^property_attribute$", "!^static_method$"]"""
display(actual)
assert actual == expected

In [ ]:
# | export


def _get_mkdocstring_config(mkdocs_path: Path) -> Tuple[int, bool]:
    """Get the mkdocstring configuration from the mkdocs.yml file.

    Args:
        mkdocs_path: The path to the mkdocs directory.

    Returns:
        A tuple containing the heading level and show category heading settings.

    Raises:
        RuntimeError: If the mkdocstrings settings cannot be read from the mkdocs.yml file.

    """
    with open((mkdocs_path / "mkdocs.yml"), "r", encoding="utf-8") as file:
        # nosemgrep: python.lang.security.deserialization.avoid-pyyaml-load.avoid-pyyaml-load
        data = yaml.load(file, Loader=yaml.Loader) # nosec: yaml_load
        mkdocstrings_config = [i for i in data["plugins"] if isinstance(i, dict) and "mkdocstrings" in i]
        if len(mkdocstrings_config) == 0:
            raise_error_and_exit(
                f"Unexpected error: cannot read mkdocstrings settings from {mkdocs_path}/mkdocs.yml file"
            )
        
        mkdocstrings_options = mkdocstrings_config[0]["mkdocstrings"]["handlers"]["python"]["options"]
        heading_level = mkdocstrings_options.get("heading_level", 2)
        show_category_heading = mkdocstrings_options.get("show_category_heading", False)
        
    return heading_level, show_category_heading

In [ ]:
_mkdocs = """
plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
        - https://docs.python.org/3/objects.inv
        options:
          heading_level: 2
          show_category_heading: true
          show_root_heading: true
          show_signature_annotations: true
          show_if_no_docstring: true
          
markdown_extensions:
- md_in_html
- pymdownx.arithmatex:
    generic: true
- pymdownx.superfences:
        custom_fences:
          - name: mermaid
            class: mermaid
            format: !!python/name:pymdownx.superfences.fence_code_format
"""

with TemporaryDirectory() as d:
    mkdocs_path = Path(d) / "mkdocs"
    mkdocs_path.mkdir(parents=True)
        
    with open((mkdocs_path / "mkdocs.yml"), "w", encoding="utf-8") as f:
        f.write(_mkdocs)

    heading_level, show_category_heading = _get_mkdocstring_config(mkdocs_path=mkdocs_path)
    actual = (heading_level, show_category_heading)
    expected = (2, True)
    
    display(actual)
    assert actual == expected

In [ ]:
_mkdocs = """
plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
        - https://docs.python.org/3/objects.inv
        options:
          show_root_heading: true
          show_signature_annotations: true
          show_if_no_docstring: true
"""

with TemporaryDirectory() as d:
    mkdocs_path = Path(d) / "mkdocs"
    mkdocs_path.mkdir(parents=True)
        
    with open((mkdocs_path / "mkdocs.yml"), "w", encoding="utf-8") as f:
        f.write(_mkdocs)

    heading_level, show_category_heading = _get_mkdocstring_config(mkdocs_path=mkdocs_path)
    actual = (heading_level, show_category_heading)
    expected = (2, False)
    
    display(actual)
    assert actual == expected

In [ ]:
_mkdocs = """
plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
        - https://docs.python.org/3/objects.inv
        options:
          heading_level: 5
          show_category_heading: false
          show_root_heading: true
          show_signature_annotations: true
          show_if_no_docstring: true
"""

with TemporaryDirectory() as d:
    mkdocs_path = Path(d) / "mkdocs"
    mkdocs_path.mkdir(parents=True)
        
    with open((mkdocs_path / "mkdocs.yml"), "w", encoding="utf-8") as f:
        f.write(_mkdocs)

    heading_level, show_category_heading = _get_mkdocstring_config(mkdocs_path=mkdocs_path)
    actual = (heading_level, show_category_heading)
    expected = (5, False)
    
    display(actual)
    assert actual == expected

In [ ]:
# | export


def get_formatted_docstring_for_symbol(
    symbol: Union[types.FunctionType, Type[Any]], mkdocs_path: Path
) -> str:
    """Recursively parses and get formatted docstring of a symbol.

    Args:
        symbol: A Python class or function object to parse the docstring for.
        mkdocs_path: The path to the mkdocs folder.

    Returns:
        A formatted docstring of the symbol and its members.

    """

    def traverse(
        symbol: Union[types.FunctionType, Type[Any]],
        contents: str,
        heading_level: int,
        show_category_heading: bool,
    ) -> str:
        """Recursively traverse the members of a symbol and append their docstrings to the provided contents string.

        Args:
            symbol: A Python class or function object to parse the docstring for.
            contents: The current formatted docstrings.
            heading_level: The base heading level set in the mkdocs config file.
            show_category_heading: The value of the show_category_heading flag set in the mkdocs config file.

        Returns:
            The updated formatted docstrings.

        """
        for x, y in getmembers(symbol):
            if not x.startswith("_"):
                if _is_method(y) and y.__doc__ is not None:
                    contents += f"{_generate_autodoc_string(y, heading_level=heading_level, show_category_heading=show_category_heading, is_root_object=False)}\n\n"
        return contents

    if symbol.__doc__ is None:
        return ""

    heading_level, show_category_heading = _get_mkdocstring_config(mkdocs_path)
    contents = _generate_autodoc_string(
        symbol, heading_level=heading_level, show_category_heading=show_category_heading
    )
    if isclass(symbol):
        contents += _filter_attributes_in_autodoc(symbol) + "\n\n"
        contents = traverse(symbol, contents, heading_level, show_category_heading)
    return contents

In [ ]:
# module_code = '''

# __all__ = ['FixtureClass']

# from typing import *
# import functools
# from abc import abstractmethod

# from fastcore.basics import patch


# def fixture_decorator(func):
#     @functools.wraps(func)
#     def wrapped_func():
#         func()
#     return wrapped_func

# class FixtureClass:
#     """Fixture documentation"""    
#     class_variable = 10
    
#     def __init__(self, attribute):
#         """__init__ documentation"""
#         self.attribute = attribute
    
#     @property
#     def property_attribute(self):
#         """property_attribute documentation"""
#         return self.attribute
    
#     @classmethod
#     def class_method(cls):
#         """class_method documentation"""
#         return cls.class_variable
    
#     @staticmethod
#     @fixture_decorator
#     def static_method():
#         """static_method documentation"""
#         return "This is a static method"
    
#     @fixture_decorator
#     def instance_method(self):
#         """instance_method documentation"""
#         return "This is an instance method"
    
#     def __str__(self):
#         """__str__ documentation"""
#         return f"MyClass instance with attribute: {self.attribute}"
    
#     @abstractmethod
#     def abstract_method(self):
#         """abstract_method documentation"""
#         pass
    
#     class NestedClass:
#         """NestedClass documentation"""
#         def nested_class_instance_method(self):
#             """nested_class_instance_method documentation"""
#             pass
        
#         @staticmethod
#         def nested_class_static_method():
#             """nested_class_static_method documentation"""
#             pass
            
#         @classmethod
#         def nested_class_class_method(cls):
#             """nested_class_class_method documentation"""
#             pass
        
#         class NestedNestedClass:
#             """NestedNestedClass documentation"""
            
#             def nested_nested_class_instance_method(self):
#                 """nested_nested_class_instance_method documentation"""
#                 pass
                
# @patch
# def patched_instance_method(self:FixtureClass, s: str) -> None: 
#     """patched_instance_method documentation"""
#     pass
    
# @patch
# def patched_class_method(cls:FixtureClass, s: str) -> None: 
#     """patched_class_method documentation"""
#     pass

# '''

# my_package = f"mypackage_{random.randint(0, 1000)}"
# expected = f'\n\n::: {my_package}.mymodule.FixtureClass\n    options:\n      filters: ["!^__str__$", "!^abstract_method$", "!^class_method$", "!^instance_method$", "!^patched_class_method$", "!^patched_instance_method$", "!^property_attribute$", "!^static_method$"]\n\n\n\n::: {my_package}.mymodule.FixtureClass.abstract_method\n    options:\n      heading_level: 6\n      show_root_full_path: false\n\n\n\n\n::: {my_package}.mymodule.FixtureClass.class_method\n    options:\n      heading_level: 6\n      show_root_full_path: false\n\n\n\n\n::: {my_package}.mymodule.FixtureClass.instance_method\n    options:\n      heading_level: 6\n      show_root_full_path: false\n\n\n\n\n::: ..patched_class_method\n    options:\n      heading_level: 6\n      show_root_full_path: false\n\n\n\n\n::: ..patched_instance_method\n    options:\n      heading_level: 6\n      show_root_full_path: false\n\n\n\n\n::: {my_package}.mymodule.FixtureClass.property_attribute\n    options:\n      heading_level: 6\n      show_root_full_path: false\n\n\n\n\n::: {my_package}.mymodule.FixtureClass.static_method\n    options:\n      heading_level: 6\n      show_root_full_path: false\n\n\n'
# with create_test_package(my_package, module_code) as d:
#     with add_tmp_path_to_sys_path(d):
#         members_with_submodules = _get_submodule_members(my_package)
#         assert my_package in members_with_submodules
#         symbols = _load_submodules(my_package, members_with_submodules)
#         assert len(symbols) > 0
#         actual = get_formatted_docstring_for_symbol(symbols[0], Path(d) / "mkdocs")
#         print(actual)
#         assert actual == expected

In [ ]:
# module_code = '''

# from typing import *

# def fixture_function(
#     arg_1: str,
#     arg_2: Union[List[str], str],
#     arg_3: Optional[int],
#     arg_4: Optional[str] = None,
# ) -> str:
#     """This is a one line description for the function

#     Args:
#         arg_1: Argument 1
#         arg_2: Argument 2
#         arg_3: Argument 3
#         arg_4: Argument 4

#     Returns:
#         The concatinated string
#     """
#     pass
# '''

# my_package = f"mypackage_{random.randint(0, 1000)}"
# expected = f"\n\n::: {my_package}.mymodule.fixture_function\n"
# with create_test_package(my_package, module_code) as d:
#     with add_tmp_path_to_sys_path(d):
#         members_with_submodules = _get_submodule_members(my_package)
#         assert my_package in members_with_submodules
#         symbols = _load_submodules(my_package, members_with_submodules)
#         assert len(symbols) > 0
        
#         actual = get_formatted_docstring_for_symbol(symbols[0], Path(d) / "mkdocs")
#         print(actual)
#         assert actual == expected